In [82]:
#解析生成json的内容
#解析决策过程的日志
import os
import json
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def paese_result_json(log_path):
    #读取json文件
    res=json.load(open(log_path,"r"))['update']
    #去除掉init_state部分
    static={}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        #print(r['tick'])
        #if r['tick']>340 and r['tick']<380:
        
        for action in actions:
            if str(action['class']) + '_' + str(action['id']) not in static:
                static[str(action['class']) + '_' + str(action['id'])]={}
            if action['action_type']=='SKILL_79':
                print(r['tick'])
            static[str(action['class']) + '_' + str(action['id'])][action['action_type']]=static[str(action['class']) + '_' + str(action['id'])][action['action_type']]+1 if action['action_type'] in static[str(action['class']) + '_' + str(action['id'])] else 1
            if 'damage' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['damage']=[]
            #print(action['class'] + '_' + str(action['id']),static[str(action['class']) + '_' + str(action['id'])]['damage'])
                #print(action)
            if 'damage' in action:
                for damage in action['damage']:
                    #print(damage)
                    static[str(action['class']) + '_' + str(action['id'])]['damage'].append(damage[2][0])                         
                    #['ATK', ['monster', 616666], [108], [108], ['Equal'], ['Default']]  [108] 是伤害值
            if 'effects' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['effects']=[]

                        
#             if action['class'] + '_' + str(action['id'])=='monster_716666':
#                 print('77777', action['class'] + '_' + str(action['id']))
            
                
            if 'effects' in action:
                if len(action['effects'])>0:
                    pass
#                     for r in action['effects']:
#                         static[str(action['class']) + '_' + str(action['id'])]['effects'].append(r[1])

        for state in states:
            if len(state)>0:
                for change in state:
                    if change[1][2]=="Hp":
                        #pass
                        if str(change[1][0]) + '_' + str(change[1][1]) not in static:
                            static[str(change[1][0]) + '_' + str(change[1][1])]={}
                        
                        if '最终HP' not in static[str(change[1][0]) + '_' + str(change[1][1])]:
                            static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=''
                        static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=change[2][1]

    for k in static.keys():
        if 'damage' not in static[k]:
            static[k]['damage']=[]
        if 'effects' not in static[k]:
            static[k]['effects']=[]
        #print(k,static[k])
        static[k]['总伤害']=sum(static[k]['damage'])
        static[k]['总伤害次数']=len(static[k]['damage'])
        static[k]['平均伤害']=sum(static[k]['damage'])/len(static[k]['damage']) if len(static[k]['damage'])>0 else 0
        static[k]['各类效果']= list(set(num for sublist in static[k]['effects'] for num in sublist))
        if 'RIGHT' not in static[k]:
            static[k]['RIGHT']=0
        if 'LEFT' not in static[k]:
            static[k]['LEFT']=0
        if 'TOP' not in static[k]:
            static[k]['TOP']=0
        if 'BOTTOM' not in static[k]:
            static[k]['BOTTOM']=0
        static[k]['总移动次数']=static[k]['RIGHT']+static[k]['LEFT']+static[k]['TOP']+static[k]['BOTTOM']
        
        
        #隐藏具体伤害
        static[k].pop('damage')
        static[k].pop('effects')

        #按照key 排序
        static[k]=dict(sorted(static[k].items(),key=lambda x:x[0]))
    print('总tick',res[-1]['tick'])
    return static


In [83]:
log_files_path = os.getcwd()+"/test_data"
    #读取这个路径下所有文件
log_files = os.listdir(log_files_path)
for log_file in log_files:
    if log_file.endswith(".json"):
        log_path = log_files_path+"/"+log_file
        static_res_json=paese_result_json(log_path)
        df=pd.DataFrame(static_res_json).T
        display(df)

72
108
144
总tick 259


,BOTTOM,LEFT,MOVE_START,RIGHT,SKILL_133,SKILL_134,SKILL_82,TOP,WAIT,各类效果,平均伤害,总伤害,总伤害次数,总移动次数,最终HP,SKILL_90,SKILL_137,SKILL_138,EFFECT_64,EFFECT_87,SKILL_100,SKILL_98,SKILL_79,SKILL_80
monster_616666,5,3,11,11,5,2,4,2,3,[],39.272727,432,11,21,522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_1316666,2,2,11,3,1,2,1,1,8,[],27.5,110,4,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_7115666,0,0,11,0,NaN,NaN,NaN,0,11,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_8115666,0,0,11,0,NaN,NaN,NaN,0,11,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_16714666,0,0,11,0,NaN,NaN,NaN,0,11,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hero_5003,0,1,2,1,NaN,NaN,NaN,4,NaN,[],63.0,315,5,6,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_516666,2,3,7,9,NaN,NaN,NaN,4,NaN,[],51.2,256,5,18,28,NaN,3,2,NaN,NaN,NaN,NaN,NaN,NaN
monster_716666,2,2,7,7,NaN,NaN,NaN,7,NaN,[],58.333333,350,6,18,114,NaN,4,2,NaN,NaN,NaN,NaN,NaN,NaN
hero_5001,3,0,7,7,NaN,NaN,NaN,11,NaN,[],51.5,309,6,21,0,NaN,NaN,NaN,7,6,3,3,NaN,NaN
hero_5002,0,0,4,10,NaN,NaN,NaN,2,NaN,[],114.0,456,4,12,0,NaN,NaN,NaN,4,NaN,NaN,NaN,3,1


In [35]:
def parse_result_json_detail(log_path):
    res = json.load(open(log_path, "r"))['update']
    # 去除掉init_state部分
    static = {}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        for i in range(len(actions)):
            if actions[i]['id']==16714666:
                if actions[i]['action_type']!='MOVE_START' and actions[i]['action_type']!='WAIT':
                    print(actions[i],states[i],r['tick'])
log_files_path = os.getcwd() + "/test_data"
    # 读取这个路径下所有文件
log_path = log_files_path + "/for_qiangye_skill_"+str(99)+".json"
log_path = log_files_path + "/for_qiangye.json"

parse_result_json_detail(log_path)

{'action_type': 'LEFT', 'move_position': [3, 1, 2], 'id': 16714666, 'class': 'monster'} [['change', ['map', 0, 0, 'Block'], [0, 2]], ['change', ['map', 0, 14, 'Block'], [2, 0]], ['change', ['monster', 16714666, 'position', 0], [4, 3]]] 22
{'action_type': 'TOP', 'move_position': [3, 1, 3], 'id': 16714666, 'class': 'monster'} [['change', ['map', 0, 0, 'Block'], [2, 0]], ['change', ['map', 0, 1, 'Block'], [0, 2]], ['change', ['monster', 16714666, 'position', 2], [2, 3]]] 22
{'action_type': 'TOP', 'move_position': [3, 1, 4], 'id': 16714666, 'class': 'monster'} [['change', ['map', 0, 1, 'Block'], [2, 0]], ['change', ['map', 0, 2, 'Block'], [0, 2]], ['change', ['monster', 16714666, 'position', 2], [3, 4]]] 22
{'action_type': 'SKILL_131', 'atk_range': [[4, 1, 3]], 'atk_position': [4, 1, 3], 'release_range': [[3, 1, 2], [3, 1, 3], [3, 1, 5], [3, 1, 6], [3, 1, 7], [4, 1, 2], [4, 1, 3], [4, 1, 4], [4, 1, 5], [4, 1, 6], [5, 1, 3], [5, 1, 4], [5, 1, 5], [6, 1, 4], [3, 1, 4]], 'damage': [['ATK', ['

In [36]:
def parse_decision_log(log_path):
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'hero' in v:
                if v['hero']==1216666:
                    if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                        if  v['stepname']=='决策树-首次查找最终选择' or  v['stepname']=='决策树-多次查找最终选择':
                            #if '判断是否满足恢复判定' in v['selection'] and v['result_len']=='True':
                                print(k,v)
                        
def static_decision_log(log_path):
    static={}
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                if  v['stepname']=='决策树-首次查找最终选择' and v['result_len']!=0:
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']][v['action']]=1
                    static[v['hero']][v['action']]=static[v['hero']][v['action']]+1
                    
                if  v['stepname']=='决策树-多次查找最终选择':
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']][v['action']]=1
                    static[v['hero']][v['action']]=static[v['hero']][v['action']]+1
                if v['stepname']=='决策树-偏好决策':
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']]['偏好_'+str(v['action'])]=1
                    static[v['hero']]['偏好_'+str(v['action'])]=static[v['hero']]['偏好_'+str(v['action'])]+1

    df=pd.DataFrame(static).T
    display(df)

In [13]:
#统计情况
static_decision_log(os.getcwd()+"/../log/e864e9e1792024_11_04_16_05_1730707558logs.json")


,偏好_等待,移动到敌人附近,移动到敌人附近并攻击,逃跑,等待,偏好_p_0_action,移动到boss
616666,2.0,2.0,4.0,2.0,NaN,NaN,NaN
1316666,15.0,2.0,4.0,4.0,8.0,NaN,NaN
7115666,30.0,3.0,10.0,7.0,13.0,NaN,NaN
8115666,30.0,2.0,6.0,5.0,20.0,NaN,NaN
16714666,30.0,3.0,9.0,5.0,16.0,NaN,NaN
5003,2.0,NaN,5.0,NaN,NaN,2.0,NaN
5001,2.0,NaN,18.0,NaN,NaN,NaN,4.0
5002,2.0,NaN,15.0,NaN,NaN,NaN,5.0
516666,2.0,NaN,2.0,2.0,NaN,NaN,NaN
716666,2.0,NaN,2.0,2.0,NaN,NaN,NaN


In [15]:

tmp_log_file = os.getcwd()+"/../log/e864e9e1792024_11_04_16_05_1730707558logs.json"
parse_decision_log(tmp_log_file)

2024-11-04 16:06:02.605288_164 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待', 'hero': 1216666}
2024-11-04 16:06:12.983813_465 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待', 'hero': 1216666}
2024-11-04 16:06:26.193245_737 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待', 'hero': 1216666}
2024-11-04 16:06:39.853002_932 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '移动到敌人附近并攻击', 'hero': 1216666}
2024-11-04 16:07:00.153222_1181 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑', 'hero': 1216666}
2024-11-04 16:07:23.500797_1432 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑', 'hero': 1216666}
2024-11-04 16:07:44.262887_1622 {'stepname': '决策树-首次查找最终选择', 'result_len': 2, 'action': '移动到敌人附近并攻击', 'hero': 1216666}
2024-11-04 16:08:13.479880_1863 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '移动到敌人附近并攻击', 'hero': 1216666}
2024-11-04 16:08:44.015054_2098 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑'